In [ ]:
# Github에서 source를 받아옵니다.
!git clone https://github.com/scpark20/AI-for-Music.git
# source directory로 변경합니다.
import os
os.chdir('AI-for-Music/source')
!pip install mido
# Google drive에서 받기 위해 gdown을 설치합니다
!pip install gdown
# maestro-npy.zip 파일을 다운로드 합니다.
!gdown https://drive.google.com/uc?id=1PCsy4C0XdO2hWedw7w6FZ39neCgAdNAi
# maestro-npy.zip 압축파일을 풉니다.
!unzip -o -qq maestro-npy.zip

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from torch.utils.data import DataLoader
from hparams import dataset_hparams, model_hparams
from model import Model

In [3]:
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

### Model Init

In [4]:
model = Model(model_hparams).cuda()
model.eval()
print(model)
print('done')

Model(
  (embedding): Embedding(390, 512)
  (rnn): LSTM(512, 1024, num_layers=3, batch_first=True, dropout=0.1)
  (out_layer): Linear(in_features=1024, out_features=390, bias=True)
)
done


### Checkpoint Download & Load

In [152]:
# Google drive에서 받기 위해 gdown을 설치합니다
!pip install gdown
# Checkpoint 파일을 다운로드 합니다.
!gdown https://drive.google.com/uc?id=1jjBMcmfZipDzR9977ViJUt6j07U_hLjX

Downloading...
From: https://drive.google.com/uc?id=1jjBMcmfZipDzR9977ViJUt6j07U_hLjX
To: /home/gaudio/ste/projects/lectures/AI-for-Music/source/checkpoint_300000
100%|████████████████████████████████████████| 284M/284M [01:06<00:00, 4.25MB/s]


In [153]:
checkpoint = torch.load('checkpoint_300000', map_location=torch.device('cpu'))    
model.load_state_dict(checkpoint['model_state_dict'])
print('done')

done


### Maestro Download & Unzip

In [156]:
!wget https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
!unzip -o -qq maestro-v3.0.0-midi.zip

--2022-01-24 02:28:09--  https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.25.80, 216.58.197.208, 172.217.26.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.25.80|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58416533 (56M) [application/octet-stream]
Saving to: ‘maestro-v3.0.0-midi.zip.2’

maestro-v3.0.0-midi 100%[===================>]  55.71M  4.01MB/s    in 14s     

2022-01-24 02:28:24 (4.02 MB/s) - ‘maestro-v3.0.0-midi.zip.2’ saved [58416533/58416533]



### Midi 파일 정보 midi_list에 담기

In [157]:
import csv 

root_dir = 'maestro-v3.0.0/'
midi_list = []
with open(root_dir + 'maestro-v3.0.0.csv') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        midi_info = {'file': root_dir + row[4],
                     'composer': row[0],
                     'title': row[1]}
        midi_list.append(midi_info)
midi_list = midi_list[1:]

for i, midi_info in enumerate(midi_list[:5]):
    print(i, midi_info)

0 {'file': 'maestro-v3.0.0/2018/MIDI-Unprocessed_Chamber3_MID--AUDIO_10_R3_2018_wav--1.midi', 'composer': 'Alban Berg', 'title': 'Sonata Op. 1'}
1 {'file': 'maestro-v3.0.0/2008/MIDI-Unprocessed_03_R2_2008_01-03_ORIG_MID--AUDIO_03_R2_2008_wav--2.midi', 'composer': 'Alban Berg', 'title': 'Sonata Op. 1'}
2 {'file': 'maestro-v3.0.0/2017/MIDI-Unprocessed_066_PIANO066_MID--AUDIO-split_07-07-17_Piano-e_3-02_wav--3.midi', 'composer': 'Alban Berg', 'title': 'Sonata Op. 1'}
3 {'file': 'maestro-v3.0.0/2004/MIDI-Unprocessed_XP_21_R1_2004_01_ORIG_MID--AUDIO_21_R1_2004_01_Track01_wav.midi', 'composer': 'Alexander Scriabin', 'title': '24 Preludes Op. 11, No. 13-24'}
4 {'file': 'maestro-v3.0.0/2006/MIDI-Unprocessed_17_R1_2006_01-06_ORIG_MID--AUDIO_17_R1_2006_04_Track04_wav.midi', 'composer': 'Alexander Scriabin', 'title': '3 Etudes, Op. 65'}


### MIDI정보를 Token으로 변환

In [168]:
from midi_utils import get_event_list, event_list_to_tokens
event_list = get_event_list(midi_list[1251]['file'])
tokens = event_list_to_tokens(event_list, dataset_hparams)
print(tokens)

[ 49 113 196 ... 344  19 389]


### Token 앞부분을 모델에 입력하고 뒤를 생성

In [169]:
from tqdm.notebook import tqdm

x = torch.from_numpy(tokens[:50]).cuda().unsqueeze(0)
state = model._get_initial_state(1)
x_new, state = model.inference(x, state, temperature=0.8)
x = torch.cat([x, x_new], dim=1)

for _ in tqdm(range(10000)):
    with torch.no_grad():
        x_new, state = model.inference(x[:, -1:], state, temperature=0.8)
        x = torch.cat([x, x_new], dim=1)
        
_x = x[0].data.cpu().numpy()
print(_x)

  0%|          | 0/10000 [00:00<?, ?it/s]

[ 49 113 196 ...   0 309   0]


### 구한 Token을 event_list로 변환하고 midi파일로 save

In [170]:
from midi_utils import tokens_to_event_list, save_event_list_to_midi_file
event_list = tokens_to_event_list(_x, dataset_hparams)
for event in event_list[:10]:
    print(event)
    
save_event_list_to_midi_file(event_list, 'AI_song_No.1.midi', speed=1.7)
print('done')

{'time': 0.98, 'type': 'note_on', 'note': 64, 'velocity': 52}
{'time': 1.02, 'type': 'note_on', 'note': 67, 'velocity': 64}
{'time': 1.06, 'type': 'note_on', 'note': 48, 'velocity': 52}
{'time': 1.06, 'type': 'note_on', 'note': 72, 'velocity': 72}
{'time': 1.1, 'type': 'note_off', 'note': 48}
{'time': 1.1800000000000002, 'type': 'note_on', 'note': 60, 'velocity': 60}
{'time': 1.2200000000000002, 'type': 'note_off', 'note': 60}
{'time': 1.2600000000000002, 'type': 'note_on', 'note': 59, 'velocity': 72}
{'time': 1.3000000000000003, 'type': 'note_off', 'note': 59}
{'time': 1.3600000000000003, 'type': 'note_on', 'note': 60, 'velocity': 64}
done


In [ ]:
from google.colab import files
files.download('AI_song_No.1.midi')